In [1]:
# load the pdf parser
import fitz
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
api_key = os.getenv("OPENAI_KEY2")
print(api_key)

sk-proj-hmErCKUzvvlgDRNZC6cRT3BlbkFJ5tRAaB2P5Nr8WQiKnyms


In [3]:
# from typing import List

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI


In [4]:
llm = ChatOpenAI(openai_api_key=api_key)

In [5]:
class elementClass(BaseModel):
    element_id:str = Field(description="the number of the given element.")
    description:str = Field(description="The text describing the element.")
    name:str = Field(description="A short text representing the element.")
    keywords:list[str] = Field(description= "some keywords of the element which can be usefull for filtering purposes.")
    value: float = Field(description="the value of the element. should be between 0 and 1.0.")
    deifficulty:str = Field(description="The difficulty of the element. should be a letter between A-J or SA or TA.")

    # setup validator
    @validator("value")
    def check_value(cls, value):
        if value < 0 or value > 1.0:
            raise ValueError("value should be between 0 and 1.0")
        return value

In [6]:
parser = JsonOutputParser(pydantic_object=elementClass)


In [7]:
template = "extract the elements from the pdf page. multiple elements can be extracted from the page."
prompt = PromptTemplate(template=template, input_variables = ["context"], partial_variables={"format_instructions": parser.get_format_instructions()})

In [8]:
chain = prompt | llm | parser

In [9]:
def load_pdf():
    loader = fitz.open("files/Women/en_2022-2024 WAG COP.pdf")
    # pages = loader.load()
    return loader 
loader = load_pdf()

In [10]:
pdf_page = loader.load_page(73).get_text()
print(pdf_page)

1.000 ʊ MO8N7S 
 
UB – Group 1 - 2 
A 
B 
C 
D 
E 
F/G 
1.103  
Glide on LB (or swing fwd on 
HB) and stoop through to rear 
support (back kip); or straddle 
cut bwd to hang on same bar 
 
 
1.203  
Reverse kip from: – glide fwd on 
LB – swing fwd on HB – stoop 
through to kip hang, back kip 
swing, seat (pike) circle bwd to 
rear support 
 
1.303 
 
1.403 
1.503 
1.603 
1.104  
Jump to hang on HB – also with 
reverse grip – kip to support 
 
1.204  
Facing HB – Jump with ½ turn 
(180°) – kip to support on HB 
 
 
Free jump with ½ turn (180°) over 
LB to hang on HB 
 
1.304  
Jump with ½ turn (180°) over 
LB – kip to support on HB 
 
 
1.404  
1.504 
1.604 
 
 



In [11]:
# response = chain.invoke({"context":pdf_page})

In [12]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

c:\Users\Loes\miniconda3\envs\routineBuilder\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Loes\miniconda3\envs\routineBuilder\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\Loes\miniconda3\envs\routineBuilder\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Loes\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disa

In [27]:
print(pdf_page.split("\n \n")[4])

1.403 
1.503 
1.603 
1.104  
Jump to hang on HB – also with 
reverse grip – kip to support 


In [28]:
# input_text = "Once upon a time"
input_ids = tokenizer.encode("get this text:"+ pdf_page.split("\n \n")[4], return_tensors='pt')

output = model.generate(input_ids, max_length=100, temperature=0.7, do_sample=True )

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [29]:
output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(output_text)

get this text:1.403 
1.503 
1.603 
1.104  
Jump to hang on HB – also with 
reverse grip – kip to support 

1.105 

1.106 

1.107 

1.108 

1.109 

1.110 

1.111 

1.112 

1.113 

1


In [32]:
from transformers import pipeline

summarizer = pipeline("summarization")
result = summarizer(pdf_page.split("\n \n")[4], max_length=50, min_length=25, do_sample=False)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Your max_length is set to 50, but your input_length is only 41. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=20)


In [34]:
result[0]

{'summary_text': ' Jumped to hang on HB – also with  reverse grip – kip to support . 1.403  \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa01.503  \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0.603  \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0: \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0\xa0receive\xa0reversal grip .'}